In [1]:
lines = """Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###..."""



In [2]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from collections import defaultdict


images = list()
image_dict = dict()

lines = open("inputs/day20.txt").read()
tiles = lines.split("\n\n")

for tile in tiles:
    if len(tile) == 0:
        continue
    name = tile.split('\n')[0].split(" ")[1][:-1]
    content = [list(x) for x in tile.split('\n')[1:]]
    content = np.array(content)=='#'
    images.append(content)
    image_dict[name] = content

In [3]:
# Determine the length of the sides of the puzzle
for w in range(100):
    if w**2 ==len(image_dict):
        SIDE_LENGTH = w


In [4]:
def get_variations(image):
    for rotation in range(4):
        for operation in [lambda x:x, lambda x:np.fliplr(x), lambda x:np.flipud(x), lambda x:np.flipud(np.fliplr(x))]:
            yield operation(np.rot90(image, k=rotation))

# Build up a dictionary with what piece can fit on what other piece            
can_fit = defaultdict(list)
for first_im in image_dict:
    for second_im in image_dict:
        if first_im == second_im: 
            continue
        fits = False
        for f1 in get_variations(image_dict[first_im]):
            for f2 in get_variations(image_dict[second_im]):
                if all(f1[:,-1] == f2[:,0]):
                    fits = True
                    break
            if fits:
                break
        if fits: 
            can_fit[first_im].append(second_im)

In [5]:
found_answer = list()

def fit_puzzle(depth, used_so_far, picked_images):        
    if depth == len(image_dict):
        print("Answer part 1", int(used_so_far[0])*int(used_so_far[SIDE_LENGTH-1])*int(used_so_far[SIDE_LENGTH*(SIDE_LENGTH-1)]) * int(used_so_far[-1]))
        found_answer.append(picked_images)
        raise ValueErrr("Found it!")
    y, x = depth//SIDE_LENGTH, depth%SIDE_LENGTH
    to_consider = image_dict.keys()
    if x > 0:
        to_consider = can_fit[used_so_far[-1]]
    for next_piece in to_consider:
        if next_piece in used_so_far: 
            continue
            
        # Corner pieces in the corner
        if (x,y) in [(0,SIDE_LENGTH-1), (SIDE_LENGTH-1,0), (0,0), (SIDE_LENGTH-1, SIDE_LENGTH-1)]:
            if len(can_fit[next_piece]) != 2:
                continue 
                
        # No edge pieces in the middle
        if 0 < x < (SIDE_LENGTH-1) and 0 < y < (SIDE_LENGTH-1):
            if len(can_fit[next_piece]) < 4:
                continue 
        if y > 0:
            above = used_so_far[(y-1)*SIDE_LENGTH + x]
            if next_piece not in can_fit[above]:
                continue
            
        for variation in get_variations(image_dict[next_piece]):
            fits_horizontal = False
            fits_vertical = False
            
            if y==0:
                fits_vertical = True
            else:
                fits_vertical = all(variation[0,:] == picked_images[(y-1,x)][-1,:])
            if x==0:
                fits_horizontal = True
            else:
                fits_horizontal = all(variation[:,0] == picked_images[(y,x-1)][:,-1])

            if fits_horizontal and fits_vertical: 
                picked_images[(y,x)] = variation
                fit_puzzle(depth+1, used_so_far + [next_piece], picked_images)
try:        
    fit_puzzle(0, [], dict())    
except Exception as e: 
    pass


Answer part 1 84116744709593


In [6]:
# Stitch the images
sea_image = np.zeros((SIDE_LENGTH*8, SIDE_LENGTH*8))
for y in range(SIDE_LENGTH):
    for x in range(SIDE_LENGTH):
        image = found_answer[0][y,x]
        sea_image[y*8:(y+1)*8,x*8:(x+1)*8] = image[1:-1, 1:-1]

In [7]:
# WATCH OUT! A SEA MONSTER!
to_search = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """
sea_monster = list()
for line in to_search.split("\n"):
    sea_monster.append([x for x in line])
to_search = np.array(sea_monster) == '#'

In [8]:
# Convolve the sea monster over the sea, and if you found a sea monster: see how rough the seas are
hs, ws = sea_image.shape
hmonster, wmonster = to_search.shape
for rotated_sea in get_variations(sea_image):
    num_monsters = 0
    is_monster = np.zeros((SIDE_LENGTH*8, SIDE_LENGTH*8))

    for y in range(hs-hmonster):
        for x in range(ws - wmonster):
            sliced = rotated_sea[y:y+hmonster, x:x+wmonster]
            if np.array_equal(np.logical_and(sliced, to_search), to_search):
                num_monsters += 1
                is_monster[y:y+hmonster, x:x+wmonster] = to_search
    if num_monsters > 0: 
        count = 0
        for y in range(hs):
            for x in range(ws):
                if rotated_sea[y,x] and not is_monster[y,x]:
                    count += 1
        print("Solution part 2", count)


Solution part 2 1957
Solution part 2 1957
